In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
import numpy as np
# disable gpu
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from tensorflow.keras.callbacks import TensorBoard


In [ ]:
%load_ext tensorboard

In [ ]:

# load train and test dataset
def load_dataset(val_split=0.2):
	# load dataset
	(trainX, trainY), (testX, testY) = mnist.load_data()
	# reshape dataset to have a single channel
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	# split train into train and validation
	n_train = int(trainX.shape[0] * (1-val_split))
	trainX, valX = trainX[:n_train], trainX[n_train:]
	trainY, valY = trainY[:n_train], trainY[n_train:]

	return trainX, trainY, valX, valY, testX, testY

def add_salt_pepper_noise(image, prob=0.1):
	"""randomly flip each pixel in the image with probability = prob
	
	each pixel has integer values in the range 0 to 255, so we flip the pixel 
	by subtracting it from 255
	"""
	flip = np.random.choice([1, 0], size=image.shape, p=[prob, 1-prob])
	# flip the pixel
	new_image = np.where(flip==1, 255-image, image)
	return new_image

def add_h_line(image, n_lines=1):
	"""add n_lines horizontal lines to the image"""
	for i in range(n_lines):
		# choose a random line
		line = np.random.randint(0, image.shape[0])
		# add the line
		image[line, :] = 255
	return image

def add_v_line(image, n_lines=1):
	"""add n_lines vertical lines to the image"""
	for i in range(n_lines):
		# choose a random line
		line = np.random.randint(0, image.shape[1])
		# add the line
		image[:, line] = 255
	return image

def add_noise(image, prob_salt_pepper=0.001, prob_h_line=0.1, prob_v_line=0.1):
	"""add salt and pepper noise and horizontal and vertical lines to the image"""
	image = add_salt_pepper_noise(image, prob=prob_salt_pepper)
	n_h_lines = np.random.binomial(1, prob_h_line)
	image = add_h_line(image, n_lines=n_h_lines)
	n_v_lines = np.random.binomial(1, prob_v_line)
	image = add_v_line(image, n_lines=n_v_lines)
	return image.copy()

# scale pixels
def prep_pixels(datasets, *args):
	new_datasets = []
	for dataset in datasets:
		# add noise to the images
		new_dataset = np.zeros(dataset.shape)
		for i in range(len(dataset)):
			new_dataset[i] = add_noise(dataset[i], *args)
		# convert from integers to floats
		new_dataset = new_dataset.astype('float32')
		# normalize to range 0-1
		new_dataset = new_dataset / 255.0
		new_datasets.append(new_dataset.copy())
	# return normalized images
	return new_datasets
 


In [ ]:
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model
 
# run the test harness for evaluating a model
def run_test_harness():
	# add TensorBoard callback
	
	import datetime

	log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
	tensorboard_callback = TensorBoard(
		log_dir=log_dir, 
		update_freq='batch', 
		histogram_freq=1,
		)
	# load dataset
	_trainX, trainY, _valX, valY, _testX, testY = load_dataset(val_split=0.1)
	
	# keep a subset of the training data for validation
	# prepare pixel data
	trainX, valX, testX = prep_pixels(
		datasets=(_trainX, _valX, _testX),
		)

	# define model
	model = define_model()
	# add early stopping
	from tensorflow.keras.callbacks import EarlyStopping
	early_stopping = EarlyStopping(
		monitor='val_loss',
		patience=10,
		verbose=1,
		mode='auto',
		restore_best_weights=True,
		)
		
	# fit model
	model.fit(
		trainX, 
		trainY, 
		epochs=50,
		batch_size=64,
		callbacks=[tensorboard_callback, early_stopping],
		validation_data=(valX, valY),
		)
	# save model
	model.save('new_model.h5')
 
# entry point, run the test harness
run_test_harness()

In [ ]:
# step by step
# load dataset
_trainX, trainY, _valX, valY, _testX, testY = load_dataset()
# prepare pixel data
trainX, valX, testX = prep_pixels([_trainX, _valX, _testX])

# display example from the dataset
from matplotlib import pyplot
for i in range(16):
    # define subplot
    pyplot.subplot(4, 4, i+1)
    # turn off axis
    pyplot.axis('off')
    # plot raw pixel data
    pyplot.imshow(_trainX[i], cmap='gray_r')

In [ ]:
# show first few images after the noise is added
for i in range(16):
    # define subplot
    pyplot.subplot(4, 4, i+1)
    # turn off axis
    pyplot.axis('off')
    # plot raw pixel data
    pyplot.imshow(trainX[i], cmap='gray_r')

In [ ]:
# show first few images along with the correct and predicted labels
from tensorflow.keras.models import load_model
model = load_model('media/classify/new_model.h5')
dataset = testX
labels = testY # one hot encoded
found = 0
square = 5
for i in range(dataset.shape[0]):
    yhat = model.predict(dataset[i].reshape(1, 28, 28, 1))
    if np.argmax(yhat) == np.argmax(labels[i]):
        color = 'green'
        continue
    else:
        color = 'red'
    # define subplot
    pyplot.subplot(square, square, found+1)
    # turn off axis
    pyplot.axis('off')
    # plot raw pixel data
    pyplot.imshow(dataset[i], cmap='gray_r')
    # predict the label
    # convert one hot to label
    label = np.argmax(yhat)
    # display also the correct label as text in black
    pyplot.text(0, 0, f"{np.argmax(labels[i])}", color='black')
    # display the predicted label as text in the color defined above
    pyplot.text(0, 7, f"{label}", color=color)
    found += 1
    if found == square**2:
        break




# Convert to ONNX

In [ ]:
import tensorflow as tf
import tf2onnx

model = tf.keras.models.load_model('media/classify/digits_model.h5')

In [ ]:
input_signature = [
    tf.TensorSpec([None, 28, 28, 1], dtype=tf.float32)
]
onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=input_signature,
    opset=13,
    output_path='media/classify/digits_model.onnx'
)

In [ ]:
import onnxruntime
import numpy as np
from tensorflow.keras.datasets import mnist

# load the MNIST dataset
(_, _), (testX, testY) = mnist.load_data()

# select a random image from the test set
idx = np.random.randint(0, testX.shape[0])
image = testX[idx]
print(image.shape)
# preprocess the image
image = image.astype('float32') / 255.0
image = np.expand_dims(image, axis=0)
image = np.expand_dims(image, axis=-1)
print(image.shape)

# load the ONNX model
sess = onnxruntime.InferenceSession('media/classify/digits_model.onnx')

# run the model on the image
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
pred = sess.run([output_name], {input_name: image})

# print the predicted label
print(f"Predicted label: {np.argmax(pred)}")
# print the true label
print(f"True label: {testY[idx]}")


In [ ]:
# time the ONNX model
%timeit sess.run([output_name], {input_name: image})